# <center>ECSE 415 - Assignment 2</center>
  ### <center>Patrick Erath - 260719203 </center>

# Question3 - Object Detection

## 3.1 - Training

### 3.1.1 - Resize Images

In [28]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

images = []

# TODO: check if grayscale is correct for HoG

# Read in each image, convert to Gray and resize, then append to list
for i in range(1,15):
    img_temp = cv2.imread("./car"+str(i)+".jpg")
    img_temp = cv2.cvtColor(img_temp, cv2.COLOR_BGR2GRAY) 
    img_temp = cv2.resize(img_temp, (128,128))
    images.append(img_temp)

### 3.1.2 - Compute HoG features

In [29]:
# Make a function which takes list of images as arguments
# and delivers list of HoG features as output. 

def hog_features(imgs_arr, cell_size=(4,4), block_size=(2,2), nbins=9):
    hog_feats_arr = []
    count = 0
    for img in imgs_arr:
        # Check window size
        if(img.shape[0]%16!=0 or img.shape[1]%16!=0):
            raise Exception('Invalid Image Size')
        else:
            # Compute window size
            win_XY = img.shape[0] // cell_size[0] * cell_size[1]
            
            # Compute blocks
            block_XY = block_size[0] * cell_size[0]
            
            # Create HoG object
            hog = cv2.HOGDescriptor(_winSize = (win_XY, win_XY),
                                    _blockSize = (block_XY, block_XY),
                                    _blockStride = (cell_size[1], cell_size[0]),
                                    _cellSize = (cell_size[1], cell_size[0]),
                                    _nbins = nbins
                                   )
            
            # Compute number of cells 
            n_cells = (img.shape[0] // cell_size[0], img.shape[1] // cell_size[1])
            
            # Compute HoG features
            hog_feats = hog.compute(img) \
                            .reshape(n_cells[1] - block_size[1] + 1,
                                     n_cells[0] - block_size[0] +1, 
                                     block_size[1], block_size[0], nbins) \
                            .transpose((1, 0, 3, 2, 4))
            
            hog_feats_arr.append(hog_feats)
            
    return hog_feats_arr
            
feats = hog_features(images)

### 3.1.3 - Calculate mean feature map across training images

In [41]:
# Map the HoGs
# Do we iterate over each HoG and take the average??

# Should still have 4 blocks

# Linearlize feats

block_1_sum = []


    block_1_sum += normalized_feats[0][:,:,0,0,0]


[[5.35033084e-02 1.46616742e-01 1.42333865e-01 1.60894781e-01
  2.45516971e-01 4.74547483e-02 1.99727207e-01 2.36317545e-01
  1.07614309e-01 2.33684540e-01 2.31868342e-01 2.16561303e-01
  8.39276835e-02 2.87064668e-02 4.06754948e-02 1.03543187e-02
  2.47658081e-02 8.05134792e-03 6.63416786e-03 1.32631268e-02
  6.98653236e-03 3.64310481e-03 4.23123129e-03 1.01354174e-01
  3.39979455e-02 8.26376304e-02 9.06031504e-02 2.03372478e-01
  8.98165107e-02 7.89745376e-02 1.60013050e-01 2.89990976e-02
  1.48028925e-01 5.07450756e-03 1.17991291e-01 1.82075784e-01
  8.33968520e-02 2.81534761e-01 2.34931692e-01 6.57822266e-02
  4.16314676e-02 2.18115076e-01 2.35786274e-01 1.90731376e-01
  1.27078056e-01 9.46671590e-02 5.88926971e-02 9.27685294e-03
  2.07671411e-02 3.39724123e-02 2.73194606e-03 2.20121932e-03
  0.00000000e+00 7.54031120e-03 2.08310299e-02 1.46410316e-02
  2.15330962e-02 8.07749853e-03 1.15800545e-01 6.28968030e-02
  5.44324070e-02 5.04738614e-02 1.09856695e-01 6.47428110e-02
  2.9065

IndexError: too many indices for array

### 3.1.4 - Repeat steps 3.1.2 - 3.1.3 for images flipped vertically

In [ ]:
imgs_flip = []

# Flip image about the vertical axis -> ie across y-axis
for img in images:
    imgs_flip.append(cv2.flip(img, 1))
    
feats_flipped = hog_features(imgs_flip)

# TODO: CHECK calculation for mean feature map

height, width = feats[0][:,:,0,0,0].shape
blocks_avg_rotated = []

# Iterate over each block (0,0), (0,1), (1,0) and (1,1)
# Iterate over each pixel, and find the average of that pixel for
# the 14 trained image
# Final array is a nested array with the average pixel for each block
for l in range(2):
    for k in range(2):
        block_avg = np.zeros((height, width))
        for h in range(height):
            pixel_sum = 0
            for w in range(width):
                for i in range(len(feats_flipped)):
                    pixel_sum = feats_flipped[i][:,:,k,l,0][h][w]
                pixel_avg = pixel_sum / (len(feats))
                block_avg[w][h]= pixel_avg
                
        blocks_avg_rotated.append(block_avg)

### 3.1.5 - Display 9 orientation channels for first block

In [ ]:
normalized_array = np.reshape(blocks_avg[0], (1,31*31))

plt.hist(normalized_array[0], 9, color='b')
plt.title("Orientation Channels for first block")
plt.show()

## 3.2 - Testing

### 3.2.1 - Extract overlapping windows from test image

In [ ]:
import time # TODO: REMOVE

test_img = cv2.imread('test.jpg')
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

# Step size is the "step" (or over lap) when sliding 
# an overlapping 128x128 window across the image
stepSize = 32
winX = (test_img.shape[1] // stepSize) * stepSize
winY = (test_img.shape[0] // stepSize) * stepSize

test_img = cv2.resize(test_img, (winX, winY))

windows_array = []

def sliding_window(image, stepSize, windowSize=(128,128)):
    # Sliding the window across the image
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])


# 1. Create a sliding window by sliding a 128x128 window across the image
for (x, y, window) in sliding_window(test_img, stepSize):
    if window.shape[0] != 128 or window.shape[1] != 128:
        continue
    else:
        # Append the given window to a window array
        windows_array.append(window)

### 3.2.2 - Compute HoG features for each window 

In [ ]:
# NOTE: Resizing the windows to 128x128 is not necessary because 
# a sliding window of 128x128 was used. Thus the windows are already
# of size 128x128

window_feats = hog_features(windows_array)

### 3.2.3 - Compute Euclidean Distance  

In [ ]:
# Compute the euclidean distance between the feature map of each
# window and the mean feature map of training images


# For non-flipped images

# Eucilidean distance only makes sense if its RGB ? 
3 

In [ ]:
# Iterate over each block (0,0), (0,1), (1,0) and (1,1)
# Iterate over each pixel, and find the average of that pixel for
# the 14 trained image
# Final array is a nested array with the average pixel for each block
for l in range(2):
    for k in range(2):
        block_avg = np.zeros((height, width))
        for h in range(height):
            pixel_sum = 0
            for w in range(width):
                for i in range(len(feats)):
                    pixel_sum = feats[i][:,:,k,l,0][h][w]
                pixel_avg = pixel_sum / (len(feats))
                block_avg[w][h]= pixel_avg
                
        blocks_avg.append(block_avg)